In [1]:
import os 
import pandas as pd
import numpy as np
import subprocess
import sys
import shutil
from distutils.dir_util import copy_tree
import distutils
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

%matplotlib notebook

# load up dependencies and functions, Note pyGSSHA_functions.py and  GSSHA.exe MUST be in kernal's working dir
%run pyGSSHA_functions.py

# Functions for processing GSSHA flood "map" columns 
Pulling out the depth information from the Max flood gfl file 

In [ ]:

###### Function to save the max flood output file into an Arc RASTER. ######  

# Note that someday I should probably pull the NumRowsCells and cols cells directly from the gfl file but whatever for now
# And should fully abstract this and put it in the PyGSSHA functions file 

def WMS_Max_Flood_File_to_ASCii(gfl_file, NumRowsCells, NumCols_Cells, NumCELLS,
                           cellsize, xll, yll, no_data_val, Save_filename, Save_FilePlace):
    
    NumCELLS = NumRowsCells* NumCols_Cells

    arr = np.genfromtxt(gfl_file, skip_header=8, delimiter=',')    # delimit with a comma to keep it from using spaces...
    arr = arr[:-1]   # Cut off the last row which says ENDDS
    arr_activecells = arr[0:NumCELLS]
    arr_datacells   =  arr[NumCELLS:]

    gridmo = np.reshape(arr_datacells, (NumRowsCells, NumCols_Cells))

    # plot it in python 
    fig, ax = plt.subplots(figsize=(8, 3))
    plt.imshow(gridmo, cmap='gray')      # Plot the data using imshow with gray colormap

    # Save the ASCii File 
    headerstring       = bytes('NCOLS %d\nNROWS %d\nXLLCENTER %f\nYLLCENTER %f\nCELLSIZE %f\nNODATA_value %f\n' % 
        (gridmo.shape[1], gridmo.shape[0], xll, yll, cellsize, no_data_val), 'UTF-8')
    with open(os.path.join(Save_FilePlace, "{}.asc".format(Save_filename)),'wb') as fout:
        fout.write(headerstring)
        np.savetxt(fout,gridmo,'%5.2f')

    # Save the projection file for ArcGIS (Projection might need to be changed depending on WMS projection?) 
    epsg = 'PROJCS["WGS_1984_UTM_Zone_4N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-159],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AUTHORITY["EPSG","32604"]],VERTCS["Local",VDATUM["Local"],PARAMETER["Direction",1.0],UNIT["Meter",1.0]]'
    with open(os.path.join(Save_FilePlace, "{}.prj".format(Save_filename)), "w") as prj:
        prj.write(epsg)
        prj.close()
        
    print("Saved {}.asc at {}".format(Save_filename, Save_FilePlace))
    
    return gridmo
    
    
######## Input Params  ###############
# Input Max flood grid file from WMS 
gfl_file = os.path.join('..', "Junk", 'Lu_testnewgrid3.gfl')

# Cell geometry from the WMS model 
NumRowsCells  = 63
NumCols_Cells = 146


# ASCii geometry, from the Lower Left corner of the model cell 
cellsize = 50
xll = 753486 + (cellsize/2)    # the ASC is based on the center of the Lower Left cell, whereas its easiest to see the corner
yll  = 2312266 + (cellsize/2)  
no_data_val = -99999

Save_filename = "testascfile11"
Save_FilePlace = os.path.join(".")

# Run the to ASC function
gridmo = WMS_Max_Flood_File_to_ASCii(gfl_file, NumRowsCells, NumCols_Cells, NumCELLS,
                           cellsize, xll, yll, no_data_val, Save_filename, Save_FilePlace)

### Function for Processing the TS depth Maps 

Notes: 
some key cells that flood seem to be
- gridmo[6,135]   = the intersection at kehekili Hwy and Waiehu beach rd (one cell N of intersection)
- gridmo[5,142] to gridmo[5,145]   = main flooding at outlet in the Maluhia church neighborhood, church is in gridmo[5,145] 
- And they key, gridmo[22,110] is the USGS stream gauge 

In [ ]:
dep_file  =  os.path.join('.', "RUN", '{}.dep'.format(PrjName))

# Cell geometry from the WMS model 
NumRowsCells        = 63
NumCols_Cells       = 146
NumTS_to_Process    = 24

Save_filename       = "{}_dep".format(PrjName)
Save_FilePlace      = os.path.join(".", "Figures/Dep_maps")

# ASCii geometry, from the Lower Left corner of the model cell 
cellsize = 50
xll = 753486 + (cellsize/2)    # the ASC is based on the center of the Lower Left cell, whereas its easiest to see the corner
yll  = 2312266 + (cellsize/2)  
no_data_val = -99999


def Process_TS_Flood_Files_to_ASCiis(dep_file, NumRowsCells, NumCols_Cells, 
                                     cellsize, xll, yll, no_data_val, Save_filename, Save_FilePlace, SAVE=False):
    
    arr = np.genfromtxt(dep_file, skip_header=7, delimiter=',', dtype=str) 
    arr = arr[:-1]   # Cut off the last row which says ENDDS
    
    NumCELLS = NumRowsCells* NumCols_Cells
    
    # This is a constant variable for each map, which uses the number of cells to determine how many rows to peel off for each timestep
    NumRowsUnit = (NumCELLS*2)+1 # *2 is because the file contains both the mask and values every time, +1 is for the TS line  

    for unit in range(0, NumTS_to_Process, 1):
        startslice = unit * NumRowsUnit
        endslice  = startslice + NumRowsUnit    
        arrrr = arr[startslice:endslice]

        TS = arrrr[0]
        arr_activecells = arrrr[1:NumCELLS+1].astype(float)
        arr_datacells   =  arrrr[NumCELLS+1:].astype(float)

        gridmo = np.reshape(arr_datacells, (NumRowsCells, NumCols_Cells))
        

        ## plot it in python 
        #fig, ax = plt.subplots(figsize=(8, 3))
        #plt.imshow(gridmo, cmap='gray')      # Plot the data using imshow with gray colormap

        #print("{} - Flood _depth at church is {}m, at gauge is {}m".format(TS, gridmo[5,145], gridmo[22,110]))
        
        if SAVE: 
            
            Save_filename_ts = Save_filename+"_"+TS.split(" ")[2][:4]
            
            # Save the ASCii File 
            headerstring       = bytes('NCOLS %d\nNROWS %d\nXLLCENTER %f\nYLLCENTER %f\nCELLSIZE %f\nNODATA_value %f\n' % 
                (gridmo.shape[1], gridmo.shape[0], xll, yll, cellsize, no_data_val), 'UTF-8')
            with open(os.path.join(Save_FilePlace, "{}.asc".format(Save_filename_ts)),'wb') as fout:
                fout.write(headerstring)
                np.savetxt(fout,gridmo,'%5.2f')

            # Save the projection file for ArcGIS (Projection might need to be changed depending on WMS projection?) 
            epsg = 'PROJCS["WGS_1984_UTM_Zone_4N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-159],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AUTHORITY["EPSG","32604"]],VERTCS["Local",VDATUM["Local"],PARAMETER["Direction",1.0],UNIT["Meter",1.0]]'
            with open(os.path.join(Save_FilePlace, "{}.prj".format(Save_filename_ts)), "w") as prj:
                prj.write(epsg)
                prj.close()

            #print("Saved {}.asc at {}".format(Save_filename_ts, Save_FilePlace))   
    
    
    
Process_TS_Flood_Files_to_ASCiis(dep_file, NumRowsCells, NumCols_Cells,  
                                 cellsize, xll, yll, no_data_val, Save_filename, Save_FilePlace, SAVE=True,)

### Modify above to produce TS of depth at a given cell location 

In [ ]:
dep_file  = os.path.join('..', "Junk", 'Lu_testnewgrid3.dep')

# Cell geometry from the WMS model 
NumRowsCells  = 63
NumCols_Cells = 146

# Timestep stuff
NumTS_to_Process    = 24
Timestep_len_mins   = 60
Start_DateTime      = "2023-01-20 22:06"


Cell_to_Extract = [22, 110]

def Process_dep_maps_to_DepthTS_oneCell(dep_file, NumRowsCells, NumCols_Cells, 
                                       NumTS_to_Process, Timestep_len_mins, Start_DateTime):  

    # Create containers to house data throughout the loops 
    #Depth_Timeseries_df = pd.DataFrame(columns=["Timestamp_txt", "DateTime", "Cell_Depth_m"])
    TSlist = []; DateTimeList = []; DepthList = []

    NumCELLS = NumRowsCells* NumCols_Cells
    
    arr = np.genfromtxt(dep_file, skip_header=7, delimiter=',', dtype=str)
    arr = arr[:-1]   # Cut off the last row which says ENDDS
    
    # This is a constant variable for each map, which uses the number of cells to determine how many rows to peel off for each timestep
    NumRowsUnit = (NumCELLS*2)+1 # *2 is because the file contains both the mask and values every time, +1 is for the TS line  
    # The start of the run in datetime 
    datetime_Start = datetime.strptime(Start_DateTime, '%Y-%m-%d %H:%M')
    
    # For each "timestamped map" this will extract the column data, throw away the "active cells mask" half, and give a np array of the map
    for unit in range(0, NumTS_to_Process, 1):
        startslice = unit * NumRowsUnit           # Start of the desired data unit
        endslice  = startslice + NumRowsUnit      # End of desired data unit
        arrrr = arr[startslice:endslice]          # a 1-d array containing each timesteps data

        TS = arrrr[0]                             # the text of the Timestep  
        arr_activecells = arrrr[1:NumCELLS+1].astype(float)  # These are the "active cells mask" part (trash)
        arr_datacells   =  arrrr[NumCELLS+1:].astype(float)  # This is the actual data we want 

        gridmo = np.reshape(arr_datacells, (NumRowsCells, NumCols_Cells))  # turn 1-d silly data into useful 2d data
        
        # Calculate the depth at the desired cell 
        DepthAtCell = gridmo[Cell_to_Extract[0], Cell_to_Extract[1]]
        
        # Calculate the actual time stamp from the start time and the delta time of each step 
        
        datetime_TS = datetime_Start + timedelta(minutes=Timestep_len_mins*unit)
        
        # Put the pertinant data in lists
        TSlist.append(TS)
        DateTimeList.append(datetime_TS)
        DepthList.append(DepthAtCell)
        
    # Create dataframe of pertinant data 
    Depth_Timeseries_df = pd.DataFrame({"Timestamp_txt":TSlist, "DateTime":DateTimeList, "Cell_Depth_m":DepthList})
    Depth_Timeseries_df.set_index("DateTime", inplace=True)
    return Depth_Timeseries_df



# Run it!
Depth_Timeseries_df = Process_dep_maps_to_DepthTS_oneCell(dep_file, NumRowsCells, NumCols_Cells, 
                                       NumTS_to_Process, Timestep_len_mins, Start_DateTime)
# Plot it
fig, ax = plt.subplots(figsize=(8, 3))
ax.plot(Depth_Timeseries_df['Cell_Depth_m'], c='b', alpha=0.4, label="Rainfall ")   # Plot rainfall

In [ ]:
dep_file  = os.path.join('..', "Junk", 'Lu_testnewgrid3.dep')

# Cell geometry from the WMS model 
NumRowsCells  = 63
NumCols_Cells = 146

# Timestep stuff
NumTS_to_Process    = 24
Timestep_len_mins   = 60
Start_DateTime      = "2023-01-20 22:06"


Cell_to_Extract = [22, 110]

def Process_dep_maps_to_DepthTS_oneCell(dep_file, Cell_to_Extract, NumRowsCells, NumCols_Cells, 
                                       NumTS_to_Process, Timestep_len_mins, Start_DateTime):  

    # Create containers to house data throughout the loops 
    #Depth_Timeseries_df = pd.DataFrame(columns=["Timestamp_txt", "DateTime", "Cell_Depth_m"])
    TSlist = []; DateTimeList = []; DepthList = []

    NumCELLS = NumRowsCells* NumCols_Cells
    
    arr = np.genfromtxt(dep_file, skip_header=7, delimiter=',', dtype=str)
    arr = arr[:-1]   # Cut off the last row which says ENDDS
    
    # This is a constant variable for each map, which uses the number of cells to determine how many rows to peel off for each timestep
    NumRowsUnit = (NumCELLS*2)+1 # *2 is because the file contains both the mask and values every time, +1 is for the TS line  
    # The start of the run in datetime 
    datetime_Start = datetime.strptime(Start_DateTime, '%Y-%m-%d %H:%M')
    
    # For each "timestamped map" this will extract the column data, throw away the "active cells mask" half, and give a np array of the map
    for unit in range(0, NumTS_to_Process, 1):
        startslice = unit * NumRowsUnit           # Start of the desired data unit
        endslice  = startslice + NumRowsUnit      # End of desired data unit
        arrrr = arr[startslice:endslice]          # a 1-d array containing each timesteps data

        TS = arrrr[0]                             # the text of the Timestep  
        arr_activecells = arrrr[1:NumCELLS+1].astype(float)  # These are the "active cells mask" part (trash)
        arr_datacells   =  arrrr[NumCELLS+1:].astype(float)  # This is the actual data we want 

        gridmo = np.reshape(arr_datacells, (NumRowsCells, NumCols_Cells))  # turn 1-d silly data into useful 2d data
        
        # Calculate the depth at the desired cell 
        DepthAtCell = gridmo[Cell_to_Extract[0], Cell_to_Extract[1]]
        
        # Calculate the actual time stamp from the start time and the delta time of each step 
        
        datetime_TS = datetime_Start + timedelta(minutes=Timestep_len_mins*unit)
        
        # Put the pertinant data in lists
        TSlist.append(TS)
        DateTimeList.append(datetime_TS)
        DepthList.append(DepthAtCell)
        
    # Create dataframe of pertinant data 
    Depth_Timeseries_df = pd.DataFrame({"Timestamp_txt":TSlist, "DateTime":DateTimeList, "Cell_Depth_m":DepthList})
    Depth_Timeseries_df.set_index("DateTime", inplace=True)
    
    return Depth_Timeseries_df



# Run it!
Depth_Timeseries_df = Process_dep_maps_to_DepthTS_oneCell(dep_file, NumRowsCells, NumCols_Cells, 
                                       NumTS_to_Process, Timestep_len_mins, Start_DateTime)
# Plot it
fig, ax = plt.subplots(figsize=(8, 3))
ax.plot(Depth_Timeseries_df['Cell_Depth_m'], c='b', alpha=0.4, label="Rainfall ")   # Plot rainfall